# Tensorflow Basic - tf.data

## tf.data
- tensorflow 2.0 에서 새롭게 추가된 모듈
- tf.placeholder, tf.feed_dict를 통해 데이터를 처리했던 불편한 방식을 극복

In [0]:
import os
import tensorflow as tf
import numpy as np

from tensorflow.keras import preprocessing # 텍스트 데이터 전처리를 위해 import

## 전처리

In [0]:
samples = ['너 오늘 이뻐 보인다',
         '나는 오늘 기분이 더러워',
         '끝내주는데, 좋은 일이 있나봐',
         '나 좋은 일이 생겼어',
         '아 오늘 진짜 짜증나',
         '환상적인데, 정말 좋은거 같아']

label = [[1],[0],[1],[1],[0],[1]] # 라벨 임의 설정 

In [0]:
# 텍스트로 구성된 데이터에서 각 단어를 단어의 인덱스로 변경(수치화)
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index

In [5]:
print('수치화한 텍스트 데이터 : \n',sequences)
print('각 단어의 인덱스 : \n',word_index)
print('라벨 : ',label)

수치화한 텍스트 데이터 : 
 [[4, 1, 5, 6], [7, 1, 8, 9], [10, 2, 3, 11], [12, 2, 3, 13], [14, 1, 15, 16], [17, 18, 19, 20]]
각 단어의 인덱스 : 
 {'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}
라벨 :  [[1], [0], [1], [1], [0], [1]]


In [6]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label)) # 두 데이터를 묶어준다
iterator = dataset.make_one_shot_iterator() # 데이터를 하나씩 사용할 수 있도록 한다
next_data = iterator.get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [7]:
#iterator를 이용해 하나씩 출력
with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError: # 더 이상 불러올 데이터가 없으면 break
      break

(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))


## 기능 1. Batch

In [24]:
# batch 사용 : 전체 데이터를 주어진 크기 만큼 분할해서 사용
BATCH_SIZE = 2

dataset = tf.data.Dataset.from_tensor_slices((sequences,label))
dataset = dataset.batch(BATCH_SIZE) # dataset의 batch 설정
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data)) # batch size가 2이기 때문에 2개 씩 출력
    except tf.errors.OutOfRangeError:
      break

(array([[4, 1, 5, 6],
       [7, 1, 8, 9]], dtype=int32), array([[1],
       [0]], dtype=int32))
(array([[10,  2,  3, 11],
       [12,  2,  3, 13]], dtype=int32), array([[1],
       [1]], dtype=int32))
(array([[14,  1, 15, 16],
       [17, 18, 19, 20]], dtype=int32), array([[0],
       [1]], dtype=int32))


## 기능 2. Shuffle

In [25]:
# shuffle 사용 : 데이터가 섞여서 출력 -> 모델 성능 향상에 사용
dataset = tf.data.Dataset.from_tensor_slices((sequences,label))
dataset = dataset.shuffle(len(sequences)) # 데이터의 길이를 인자로 주면 전체 데이터에 대해 셔플
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))


## 기능 3. Epoch

In [26]:
# epoch 사용 : 전체 데이터를 몇 번 반복해서 불러올것인지 설정
EPOCH = 2

dataset = tf.data.Dataset.from_tensor_slices((sequences,label))
dataset = dataset.repeat(EPOCH) # 데이터의 길이를 인자로 주면 전체 데이터에 대해 셔플
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))
(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))


## 기능 4. Mapping

In [28]:
# mapping 사용 : 라벨을 제외한 나머지 데이터를 하나의 입력값으로 묶기 위해 사용 -> 모델에 따라 입력값이 두 개 이상일 때 사용
def mapping_fn(X,Y=None):
  input = {'x':X}
  label = Y
  return input, label

dataset = tf.data.Dataset.from_tensor_slices((sequences,label))
dataset = dataset.map(mapping_fn)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

({'x': array([4, 1, 5, 6], dtype=int32)}, array([1], dtype=int32))
({'x': array([7, 1, 8, 9], dtype=int32)}, array([0], dtype=int32))
({'x': array([10,  2,  3, 11], dtype=int32)}, array([1], dtype=int32))
({'x': array([12,  2,  3, 13], dtype=int32)}, array([1], dtype=int32))
({'x': array([14,  1, 15, 16], dtype=int32)}, array([0], dtype=int32))
({'x': array([17, 18, 19, 20], dtype=int32)}, array([1], dtype=int32))


## 위 4 개의 기능을 통합

In [8]:
BATCH_SIZE = 2
EPOCH = 2

def mapping_fn(X,Y=None):
  input = {'x':X}
  label = Y
  return input, label

dataset = tf.data.Dataset.from_tensor_slices((sequences,label))
dataset = dataset.map(mapping_fn)
dataset = dataset.shuffle(len(sequences))
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.repeat(EPOCH)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

({'x': array([[ 4,  1,  5,  6],
       [12,  2,  3, 13]], dtype=int32)}, array([[1],
       [1]], dtype=int32))
({'x': array([[17, 18, 19, 20],
       [14,  1, 15, 16]], dtype=int32)}, array([[1],
       [0]], dtype=int32))
({'x': array([[ 7,  1,  8,  9],
       [10,  2,  3, 11]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
({'x': array([[ 4,  1,  5,  6],
       [17, 18, 19, 20]], dtype=int32)}, array([[1],
       [1]], dtype=int32))
({'x': array([[12,  2,  3, 13],
       [14,  1, 15, 16]], dtype=int32)}, array([[1],
       [0]], dtype=int32))
({'x': array([[ 7,  1,  8,  9],
       [10,  2,  3, 11]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
